In this tutorial we're looking at the electrokinetics feature of espresso, which allows to describe the motion of potentially charged chemical species solvated in a fluid on a continuum level. The govering equations are known as the Poisson-Nernst-Planck equation, which is the combination of the electrostatic Poisson equation and the dynamics of the chemical species described by the Nernst-Planck equation. For the advection we solve the incompressible Navier-Stokes equation. The total set of equations is given by

\begin{aligned}
\partial_{t} n_{i} &= - \vec{\nabla} \cdot \vec{j}_{i} \\
\vec{j}_{i} &= - D_{i} \vec{\nabla} n_{i} - \frac{z_{i} e}{k_{B} T} n_{i} \vec{\nabla} \phi + n_{i} \vec{u} \\
\Delta \phi &= \frac{1}{4 \pi \epsilon_{0} \epsilon_{\mathrm{r}}} \sum_{i} z_{i} e n_{i} \\
\rho (\partial_{t} \vec{u} + (\vec{u} \cdot \vec{\nabla}) \vec{u}) &= - \vec{\nabla} p + \eta \Delta \vec{u} + \sum_{i} \frac{k_{B} T}{D_{i}} \vec{j}_{i} + \vec{f}_{\mathrm{ext}} \\
\vec{\nabla} \cdot \vec{u} &= 0,
\end{aligned}

where $n_{i}$ denotes the ion density of species $i$, $\vec{j}_{i}$ the density flux, $D_{i}$ the diffusion coefficient, $z_{i}$ the valency, $e$ the elementary charge, $k_{B}$ the Boltzmann constant, $T$ the temperature, $\phi$ the electrostatic potential, $\epsilon_{0}$ the vacuum permittivity, $\epsilon_{\mathrm{r}}$ the relative permittivity, $\rho$ the fluid density, $\vec{u}$ the fluid velocity, $p$ the hydrostatic pressure, $\eta$ the dynamic viscosity, and $\vec{f}_{\mathrm{ext}}$ an external force density.

# Advection-Diffusion equation in 2D

The first system that is simulated in this tutorial is the simple advection-diffusion of a drop of uncharged chemical species in a constant velocity field. To keep the computation time small, we restrict ourselves to a 2D problem, but the algorithm is also capable of solving the 3D advection-diffusion equation. Furthermore, we can also skip solving the electrostatic-poisson equation, since there are is no charged species present. This system can be described by the diffusion-equation for the density of the chemical species, which reads

\begin{equation}
\partial_{t} n = D \Delta n - \vec{\nabla} \cdot (\vec{v} n).
\end{equation}

The fundamental solution of this partial diffential equation can easily be found in the case of a constant velocity field $\vec{v}$ and a constant diffusion coefficient $D$. For a $d$-dimensional system, the solution of an inital droplet at the origin can be written as

\begin{equation}
n(\vec{x},t) = \frac{1}{(4 \pi D t)^{d/2}} \exp \left( - \frac{(\vec{x} - \vec{v} t)^2}{4 D t} \right).
\end{equation}

After importing the necessary packages, we start by defining the necessary parameters for the simulation. Since the electrokinetics algorithm can be coupled to the lattice Boltzmann solver for the hydrodynamics, we use it's velocity field to provide a constant velocity field for the advction of the chemical species.

In [ ]:
import espressomd
import espressomd.lb
import espressomd.electrokinetics
import espressomd.shapes

espressomd.assert_features(["WALBERLA", "WALBERLA_FFT"])

import tqdm
import numpy as np

import scipy.optimize

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.quiver

import tempfile
import base64

plt.rcParams.update({'font.size': 16})

In [ ]:
BOX_L = [80, 80, 1] 
AGRID = 1.0
DIFFUSION_COEFFICIENT = 0.06
TAU = 1.0
EXT_FORCE_DENSITY = [0, 0, 0]
FLUID_DENSITY = 1.0
FLUID_VISCOSITY = 1.
FLUID_VELOCITY = [0.04, 0.04, 0.0]
KT = 1.0

RUN_TIME = 200

In [ ]:
system = espressomd.System(box_l=BOX_L)
system.time_step = TAU
system.cell_system.skin = 0.4

The algorithm we are using is a grid based one, the same as the lattice Boltzmann algorithm. Therefore we start with defining a lattice, which will define the resolution of the grid we use to simulate our problem.

In [ ]:
lattice = espressomd.lb.LatticeWalberla(
    n_ghost_layers=1, agrid=AGRID)

Next we setup the lattice Boltzmann fluid, which we only use to provide the constant velocity field for our advection and add it to the `system`-object to be intergrated when running the integrator. Note that the value for `kT` is set to 0, because we only want the constant velocity field.

In [ ]:
lb = espressomd.lb.LBFluidWalberla(
                lattice=lattice, density=FLUID_DENSITY, kinematic_viscosity=FLUID_VISCOSITY,
                tau=TAU, ext_force_density=EXT_FORCE_DENSITY, kT=0.0, seed=42)
lb[:, :, :].velocity = FLUID_VELOCITY
system.lb = lb

To use the electrokinetics-algorithm in espresso, one needs to create an instance of the `EKContainer`-object and give it both a `tau` and `solver`, which are the timestep and the poisson-solver, respectively.
Since our species is uncharged, we don't need to solve the electrostatic Poisson-equation, so we can use the placeholder-class, which is called `EKNone`.

In [ ]:
eksolver = espressomd.electrokinetics.EKNone(lattice=lattice)
system.ekcontainer = espressomd.electrokinetics.EKContainer(tau=TAU, solver=eksolver)

Adding `EKSpecies` to this container will add them to be integrated by the integrator and optionally be used to calculate the charge-field used to solve the Poisson-equation.
We create an uncharged chemical species and add it to the container to calculate the advection-diffusion. It is initialized with a density of `0.0` and only the diffusion and the advection of the species is enabled.

# Exercise:
- Create an instance of the [`espressomd.electrokinetics.EKSpecies`]() and add it to the system with [`system.ekcontainer.add()`](https://espressomd.github.io/doc/espressomd.html#espressomd.electrokinetics.EKContainer.add). 

# Hint:
- Use the defined values for `DIFFUSION_COEFFICIENT`, `KT` and `TAU`.
- Enable both `advection` and `friction_coupling`.
- Make sure to initialize the `density` with 0.0, and disable electrostatics by setting `valency` to 0.0 as well.

In [ ]:
species = espressomd.electrokinetics.EKSpecies(
    lattice=lattice, density=0.0, kT=KT,
    diffusion=DIFFUSION_COEFFICIENT, valency=0.0,
    advection=True, friction_coupling=True,
    ext_efield=[0., 0., 0.], tau=TAU)
system.ekcontainer.add(species)

To compare our simulation to the fundamental solution of the Advection-Diffusion equation, we need to approximate a delta-droplet, which can be done by having only a non-zero density at the center of the domain.

In [ ]:
system.ekcontainer[0][BOX_L[0] // 2, BOX_L[1] // 2, 0].density = 1.0

Now everything is set and we can finally run the simulation by running the integrator.

In [ ]:
system.integrator.run(RUN_TIME)

For the analytic solution we need the physical positions, which we need to calculated ourselves. This can be achieved using `np.meshgrid`. The advection is incooperated by a simple Galilei-transformation of the coordinate system.

In [ ]:
def density(pos: np.ndarray, time: int, D: float):
    dim = pos.shape[-1]
    return (4 * np.pi * D * time)**(-dim / 2) * np.exp(-np.sum(np.square(pos), axis=-1) / (4 * D * time))

pos = np.zeros((*BOX_L[:2], 2))
xpos = np.arange(-BOX_L[0] // 2, BOX_L[0] // 2)
ypos = np.arange(-BOX_L[1] // 2, BOX_L[1] // 2)

pos[..., 1], pos[..., 0] = np.meshgrid(xpos, ypos)

# add the advection shift
pos -= np.asarray(FLUID_VELOCITY[:2]) * TAU * RUN_TIME

analytic_density = density(pos=pos, time=RUN_TIME*TAU, D=DIFFUSION_COEFFICIENT)
# why is this necessary? dx=1, dt=1???
analytic_density /= np.sum(analytic_density)

To compare the results, we can plot the density as image-plots as well as only a diagonal slice of the density as a line-plot.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(15, 7))

ax1.imshow(system.ekcontainer[0][:, :, 0].density, origin="lower", vmin=0, vmax=6e-3)
ax1.set_title("simulation")

imshow = ax2.imshow(analytic_density, origin="lower", vmin=0, vmax=6e-3)
ax2.set_title("analytic")
fig.colorbar(imshow, ax=[ax1, ax2], shrink=0.8)

In [ ]:
values_diagonal = np.diagonal(system.ekcontainer[0][:, :, 0].density)
analytic_diagonal = np.diagonal(analytic_density)
positions_diagonal = np.arange(len(values_diagonal)) * np.sqrt(2)

fig = plt.figure(figsize=(8, 5))
ax = fig.gca()

ax.plot(positions_diagonal, values_diagonal, "x", label="simulation")
ax.plot(positions_diagonal, analytic_diagonal, label="analytic")

ax.set_xlabel("position")
ax.set_ylabel("density")

plt.legend()

From the plot one can see that the position of the density-peak matches well. However, one also sees a slightly larger diffusive behavior than expected. The reason is the used discretization for the advection, which introduces an artifical diffusion to the system. This is a limitation of the algorithm, which is why it cannot be applied to pure advection problems.

## Pressure-driven vs electroosmotic flow

The next system in this tutorial is a simple slit pore, as depicted in Figure 1. It consists of an infinite plate capacitor with a electrolytic solution trapped in between. The plates of the capactior carry a constant surface charge and the counterions are solvated in the liquid. This setup will cause the ions to accumulate on the surface of the plates, forming a characteristic ion density profile, which can be solved in a mean-field manner using the Poisson-Boltzmann equation. Since the system is translational symmetric in the direction parallel to the plates, the equations for parallel and orthogonal direction will decouple. This also means that applying an external electric field in direction parallel to the plates will not change the distribution of the ions, but will result in a flow of the fluid. This characteristic flow profile is called electroomsotic flow.

<figure>
    <img src='figures/schlitzpore_3d.png' alt='missing' style=\"width: 500px;\"/>
    <center>
    <figcaption>Figure 1: Slit pore system and coordinate system used for the analytical calculations.</figcaption>
    </center>
</figure>

Due to the symmetries of the system, it effectively reduces to a 1D problem. Analytically, this system can be described by the Poisson-Boltzmann equation:

$$
\partial_{x}^2 \phi(x) = \frac{1}{\epsilon_{0} \epsilon_{\mathrm{r}}} \sum_{i} z_{i} e n_{i}(x) \exp \left( -\frac{z_{i} e \phi(x)}{k_{\mathrm{B}} T} \right)
$$

where $x$ is the normal-direction of the plates. Since we will only simulate a single ion species, the counterions, the sum only has a single summand. The analytic solution of the potential is then given by:

\begin{equation}
\phi(x) = -\frac{k_{B}T}{z e} \log \left[ \frac{C^2 \epsilon_{0} \epsilon_{\mathrm{r}}}{2 k_{B}T } \cos^{-2} \left( \frac{z e C}{2 k_{B} T} x \right) \right], \qquad \text{with } \left\| \frac{z e C}{2 k_{B} T} \right\| < \frac{\pi}{2},
\end{equation}

where $C$ is an integration constant that is to be determined by the boundary conditions. The ion density follows then from the potential and reads

\begin{equation}
n(x) = \frac{C^2 \epsilon_{0} \epsilon_{\mathrm{r}}}{2 k_{B}T} \cos^{-2} \left( \frac{z e C}{2 k_{B} T} x \right).
\end{equation}

To find the integration constant we can use that the total system has to be charge neutral, i.e. the total charge on the plates is counterbalanced by the counterions. This leads to the following equation

\begin{equation}
C \tan \left( \frac{z e d}{4 k_{B} T} C \right) = - \frac{e^2}{\epsilon_{0} \epsilon_{\mathrm{r}}} \sigma,
\end{equation}

where $\sigma$ is the surface charge density of the plates. This is a transcendental equation, which can be solved numerically to find $C$. The electric field in the system is then applied in the $y$-direction of the system.
Fluid flow is described by the incompressible Navier-Stokes equation, which due to the symmetries of the system reduces to a one-dimensional problem, which reads

\begin{equation}
\frac{\partial^2 v_{y}(x)}{\partial x^2} = - \frac{\epsilon_{0} \epsilon_{\mathrm{r}} z e E C^2}{2 k_{B}T \eta} \cos^{-2}\left( \frac{q C}{2 k_{B} T} x \right).
\end{equation}

This equation can be solved analytically and the solution to it is given by

\begin{equation}
v_{y}(x) = \frac{2 \epsilon_{0} \epsilon_{\mathrm{r}} k_{B} T E}{\eta z e} \log \left( \frac{\cos \left( \frac{z e C}{2 k_{B} T} x \right)}{\cos \left( \frac{z e C}{2 k_{B} T} \frac{d}{2} \right)} \right),
\end{equation}

where $d$ denotes the distance between the two plates. Finally, the shear-stress of this problem is given by

\begin{equation}
\sigma(x) = \mu \frac{\partial v_{y}(x)}{\partial x}
\end{equation}

This, together with the density profile, will then be compared to the analytical solution in the following.

In [ ]:
system.ekcontainer = None
system.lb = None

First the parameters for the simulation are specified.

In [ ]:
AGRID = 1.0
TAU = 1.0
KT = 2.0
PERMITTIVITY = 0.28
DIFFUSION_COEFFICIENT = 0.25
VALENCY = 1.0
VISCOSITY_DYNAMIC = 0.5
DENSITY_FLUID = 1.0
SURFACE_CHARGE_DENSITY = -0.05
EXT_FORCE_DENSITY = [0.0, 0.01, 0.0]

SINGLE_PRECISION = False

padding = 1
WIDTH = 200
BOX_L = [WIDTH + 2 * padding, 1, 1]

system.cell_system.skin = 0.4
system.box_l = BOX_L
system.time_step = TAU

RUN_TIME = 600

We can now set up the electrokinetics algorithm, which follows the pattern as in the first case, starting with the LB-method.

In [ ]:
lattice = espressomd.lb.LatticeWalberla(agrid=AGRID, n_ghost_layers=1)

In [ ]:
viscosity_kinematic = VISCOSITY_DYNAMIC / DENSITY_FLUID
lbf = espressomd.lb.LBFluidWalberla(lattice=lattice, density=DENSITY_FLUID, kinematic_viscosity=viscosity_kinematic, tau=TAU, single_precision=SINGLE_PRECISION)
system.lb = lbf

Since our species are going to carry a charge now, we need to solve the electrostatic Poisson-equation. For that, we have to specify a solver.

# Exercise: 
- Setup an Poissonsolver for the electrostatic interaction and create an instance of the [EKContainer](https://espressomd.github.io/doc/espressomd.html#espressomd.electrokinetics.EKContainer) object with it and `TAU`, which is set as a member of the system-class ([`system.ekcontainer`](https://espressomd.github.io/doc/espressomd.html#espressomd.system.System.ekcontainer)).

# Hint:
- Use an [EKFFT](https://espressomd.github.io/doc/espressomd.html#espressomd.electrokinetics.EKFFT)-object as the Poisson-solver.

In [ ]:
eksolver = espressomd.electrokinetics.EKFFT(lattice=lattice, permittivity=PERMITTIVITY, single_precision=SINGLE_PRECISION)

system.ekcontainer = espressomd.electrokinetics.EKContainer(tau=TAU, solver=eksolver)

To simulate this system, we will use two different ion species. The regular counterions, which are propagating in the system, and a species we will disable the propagation of. The latter one will be stationary and be used to describe the surface charge on the plates.

In [ ]:
ekspecies = espressomd.electrokinetics.EKSpecies(lattice=lattice, density=0.0, kT=KT, diffusion=DIFFUSION_COEFFICIENT, valency=VALENCY, advection=True, friction_coupling=True, ext_efield=EXT_FORCE_DENSITY, single_precision=SINGLE_PRECISION, tau=TAU)
system.ekcontainer.add(ekspecies)

ekwallcharge = espressomd.electrokinetics.EKSpecies(lattice=lattice, density=0.0, kT=KT, diffusion=0., valency=-VALENCY, advection=False, friction_coupling=False, ext_efield=[0, 0, 0], single_precision=SINGLE_PRECISION, tau=TAU)
system.ekcontainer.add(ekwallcharge)

Now we can distribute the ions in the domain. The counterions will be initialized with a homogeneous distribution, excluding the cells used as boundaries. The surface charge density is homogenously distributed in the boundary cells.

In [ ]:
density_counterions = -2.0 * SURFACE_CHARGE_DENSITY / WIDTH
ekspecies[padding:-padding, :, :].density = density_counterions

ekspecies[:padding, :, :].density = 0.0
ekspecies[-padding:, :, :].density = 0.0

ekwallcharge[:padding, :, :].density = -SURFACE_CHARGE_DENSITY / VALENCY / padding
ekwallcharge[-padding:, :, :].density = -SURFACE_CHARGE_DENSITY / VALENCY / padding

Up to now we only setup the domain, but haven't specified yet which boundary-conditions we want to enforce. For that we can use `shapes` in espresso, at which we can specify boundary conditions. Since our geometry consists of two walls, we can use them to specify our boundary conditions.

In [ ]:
wall_left = espressomd.shapes.Wall(normal=[1, 0, 0], dist=padding)
wall_right = espressomd.shapes.Wall(normal=[-1, 0, 0], dist=-(padding + WIDTH))

At both of them we specify No-Flux boundary conditions for the ion densities, so no fluxes will be allowed to enter or leave the wall. Furthermore, we specify a zero-density boundary condition for the species and a No-Slip boundary condition for the fluid flow.

In [ ]:
for shape_obj in (wall_left, wall_right):
    ekspecies.add_boundary_from_shape(shape=shape_obj, value=[0., 0., 0.], boundary_type=espressomd.electrokinetics.FluxBoundary)
    ekspecies.add_boundary_from_shape(shape=shape_obj, value=0.0, boundary_type=espressomd.electrokinetics.DensityBoundary)
    lbf.add_boundary_from_shape(shape=shape_obj, velocity=[0., 0., 0.])

Now we can finally integrate the system and extract the ion density profile, the fluid velocity profile as well as the pressure-tensor profile.

In [ ]:
for i in tqdm.trange(100):
    system.integrator.run(RUN_TIME)

In [ ]:
mid_y = int(system.box_l[1] / (2 * AGRID))
mid_z = int(system.box_l[2] / (2 * AGRID))
density_eof = ekspecies[padding:-padding, mid_y, mid_z].density
velocity_eof = lbf[padding:-padding, mid_y, mid_z].velocity[:, 1]
pressure_tensor_eof = lbf[padding:-padding, mid_y, mid_z].pressure_tensor[:, 0, 1]

positions = (np.arange(len(density_eof)) - WIDTH / 2 + 0.5) * AGRID

For the comparison, we have to calculate the anlytic solution

In [ ]:
def transcendental_equation(c, distance, kT, sigma, valency, permittivity) -> float:
    elementary_charge = 1.0
    return c * np.tan(valency * elementary_charge * distance / (4 * kT) * c) + sigma / permittivity

solution = scipy.optimize.fsolve(func=transcendental_equation, x0=0.001, args=(WIDTH, KT, SURFACE_CHARGE_DENSITY, VALENCY, PERMITTIVITY))


def eof_density(x, c, permittivity, elementary_charge, valency, kT):
    return c**2 * permittivity / (2 * kT) / (np.cos(valency * elementary_charge * c / (2 * kT) * x))**2

def eof_velocity(x, c, permittivity, elementary_charge, valency, kT, ext_field, distance, viscosity):
    return 2 * kT * ext_field * permittivity / (viscosity * elementary_charge * valency) * np.log(np.cos(valency * elementary_charge * c / (2 * kT) * x) / np.cos(valency * elementary_charge * c / (2 * kT) * distance / 2))

def eof_pressure_tensor(x, c, elementary_charge, valency, kT, ext_field, permittivity):
    return permittivity * ext_field * c * np.tan(valency * elementary_charge * c / (2 * kT) * x)

In [ ]:
analytic_density_eof = eof_density(x=positions, c=solution, permittivity=PERMITTIVITY, elementary_charge=1.0, valency=VALENCY, kT=KT)
analytic_velocity_eof = eof_velocity(x=positions, c=solution, permittivity=PERMITTIVITY, elementary_charge=1.0, valency=VALENCY, kT=KT, ext_field=EXT_FORCE_DENSITY[1], distance=WIDTH, viscosity=VISCOSITY_DYNAMIC)
analytic_pressure_tensor_eof = eof_pressure_tensor(x=positions, c=solution, elementary_charge=1.0, valency=VALENCY, kT=KT, ext_field=EXT_FORCE_DENSITY[1], permittivity=PERMITTIVITY)

In [ ]:
fig1 = plt.figure(figsize=(16, 4.5))
fig1.suptitle("electroosmotic flow")

ax = fig1.add_subplot(131)
ax.plot(positions, density_eof, "o", mfc="none", label="simulation")
ax.plot(positions, analytic_density_eof, label="analytic")
ax.set_xlabel("x-position")
ax.set_ylabel("Counter-ion density")
ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))
ax.legend(loc="best")

ax = fig1.add_subplot(132)
ax.plot(positions, velocity_eof, "o", mfc="none", label="simulation")
ax.plot(positions, analytic_velocity_eof, label="analytic")
ax.set_xlabel("x-position")
ax.set_ylabel("Fluid velocity")
ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))
ax.legend(loc="best")

ax = fig1.add_subplot(133)
ax.plot(positions, pressure_tensor_eof, "o", mfc="none", label="simulation")
ax.plot(positions, analytic_pressure_tensor_eof, label="analytic")
ax.set_xlabel("x-position")
ax.set_ylabel("Fluid shear stress xz")
ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))
ax.legend(loc="best")

plt.tight_layout()
plt.show()

In the plots one can see that the analytic solution for the electroosmotic flow matches the simulation very well. 

To compare this result to a pressure-driven flow, we need to turn off the external electric field and enable a constant external force density on the fluid.

In [ ]:
EXT_FORCE_DENSITY = [0.0, 0.000004, 0.0]

ekspecies.ext_efield = [0.0, 0.0, 0.0]
lbf.ext_force_density = EXT_FORCE_DENSITY

This new system can be integrated again and the exact same measurement as the one before are taken.

In [ ]:
for i in tqdm.trange(50):
    system.integrator.run(RUN_TIME)

In [ ]:
density_pressure = ekspecies[padding:-padding, mid_y, mid_z].density
velocity_pressure = lbf[padding:-padding, mid_y, mid_z].velocity[:, 1]
pressure_tensor_pressure = lbf[padding:-padding, mid_y, mid_z].pressure_tensor[:, 0, 1]

Again we can compare our solution to an anlytic solution, which is known as the Poiseuille flow between infinite parallel plates.

In [ ]:
def pressure_velocity(x, distance, ext_field, viscosity):
    return ext_field / (2 * viscosity) * (distance**2 / 4 - x**2)

def pressure_pressure_tensor(x, ext_field):
    return ext_field * x

In [ ]:
analytic_velocity_pressure = pressure_velocity(x=positions, distance=WIDTH, ext_field=EXT_FORCE_DENSITY[1], viscosity=VISCOSITY_DYNAMIC)
analytic_pressure_tensor_pressure = pressure_pressure_tensor(x=positions, ext_field=EXT_FORCE_DENSITY[1])

In [ ]:
fig1 = plt.figure(figsize=(16, 4.5))
fig1.suptitle("pressure driven flow")

ax = fig1.add_subplot(131)
ax.plot(positions, density_pressure, "o", mfc="none", label="simulation")
ax.plot(positions, analytic_density_eof, label="analytic")
ax.set_xlabel("x-position")
ax.set_ylabel("counter-ion density")
ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))
ax.legend(loc="best")

ax = fig1.add_subplot(132)
ax.plot(positions, velocity_pressure, "o", mfc="none", label="simulation")
ax.plot(positions, analytic_velocity_pressure, label="analytic")
ax.set_xlabel("x-position")
ax.set_ylabel("fluid velocity")
ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))
ax.legend(loc="best")

ax = fig1.add_subplot(133)
ax.plot(positions, pressure_tensor_pressure, "o", mfc="none", label="simulation")
ax.plot(positions, analytic_pressure_tensor_pressure, label="analytic")
ax.set_xlabel("x-position")
ax.set_ylabel("fluid shear stress xz")
ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))
ax.legend(loc="best")

plt.tight_layout()
plt.show()

As one can again see, the body force on the fluid did non alter the ion-density profile, and one can observe a good match for the parabolic fluid velocity.

To see the difference between the two types of flows, we plot the simulation data together in one plot.

In [ ]:
fig1 = plt.figure(figsize=(16, 4.5))
fig1.suptitle("electroosmotic - pressure driven flow comparison")

ax = fig1.add_subplot(131)
ax.plot(positions, density_eof, "o", mfc="none", label="eof")
ax.plot(positions, density_pressure, "o", mfc="none", label="pressure")
ax.set_xlabel("x-position")
ax.set_ylabel("counter-ion density")
ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))
ax.legend(loc="best")

ax = fig1.add_subplot(132)
ax.plot(positions, velocity_eof, "o", mfc="none", label="eof")
ax.plot(positions, velocity_pressure, "o", mfc="none", label="pressure")
ax.set_xlabel("x-position")
ax.set_ylabel("fluid velocity")
ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))
ax.legend(loc="best")

ax = fig1.add_subplot(133)
ax.plot(positions, pressure_tensor_eof, "o", mfc="none", label="eof")
ax.plot(positions, pressure_tensor_pressure, "o", mfc="none", label="pressure")
ax.set_xlabel("x-position")
ax.set_ylabel("fluid shear stress xz")
ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))
ax.legend(loc="best")

plt.tight_layout()
plt.show()

Looking at the fluid velocity plot, one can see that the electroosmotic flow profile flattens significantly faster towards the center of the channel when compared to the pressure driven flow. The reason for this is the accumulation of the counterion-density towards the plates of the system, where the driving electric field causes the highest force on the fluid, which decays towards the center of the channel. In contrast, the Poiseuille-flow is driven by a constant, uniform driving force.

# Kármán vortex street with a reactive species

As a showcase for the algorithm, we are now simulating a [Kármán vortex street](https://en.wikipedia.org/wiki/K%C3%A1rm%C3%A1n_vortex_street).
Our setup consists of rigid cylinders in a fast, unsteady fluid flow, which develops to a repeating pattern of swirling vorticies behind the obstacle.
To showcase the reaction feature of the electrokinetics algorithm in espresso, we're adding several species to the system undergoing advection-diffusion, which is dominated by the downstream fluid flow in the channel.
The reaction will be included as a bulk-reaction, which means that the reaction can happen anywhere, the only requirement is that both species are present in the same lattice-cell. When this occurs, parts of these species will undergo the reaction and will turn into the product. How much of the species will transform within each timestep is determined by the respective reaction rate and the overall structure of the reaction.


In [ ]:
system.ekcontainer = None
system.lb = None

The system is again setup in the same manner as in the previous problem, defining the dimensions of the system as well as the `time_step`.

In [ ]:
BOX_L = [80, 30, 1] 
AGRID = 1.0
DIFFUSION_COEFFICIENT = 0.01
TAU = 0.01
EXT_FORCE_DENSITY = [0.6, 0, 0]
OBSTACLE_RADIUS = 5

DENSITY_FLUID = 0.5
VISCOSITY_KINEMATIC = 2.0
KT = 1.0

TOTAL_FRAMES = 200

system.time_step = TAU
system.cell_system.skin = 0.4
system.box_l = BOX_L

Since we're again using the LB/EK algorithm, we have to specify the grid, on which we want to solve our problem. For simplicity and for the sake of computation time, we restrict ourselves to a effectively 2D grid.

In [ ]:
lattice = espressomd.lb.LatticeWalberla(
    n_ghost_layers=1, agrid=AGRID)

lb = espressomd.lb.LBFluidWalberla(
                lattice=lattice, density=DENSITY_FLUID, kinematic_viscosity=VISCOSITY_KINEMATIC,
                tau=TAU, ext_force_density=EXT_FORCE_DENSITY, kT=KT, seed=42)
system.lb = lb


eksolver = espressomd.electrokinetics.EKNone(lattice=lattice)
system.ekcontainer = espressomd.electrokinetics.EKContainer(tau=TAU, solver=eksolver)

Now we can actually focus on the reactions, we want to include into our simulation. In this tutorial we choose the simple case of $A + B \rightarrow C$, which means that equal parts of the educt species $A$ and $B$ can turn into the product species $C$.
Espresso distinguishes between educts and products by the sign of the stoechiometric coefficients, where educts have negative coefficients and products positive coefficients. Intuitively this can be understood that when a reaction happens, the density of the educts will decrease, hence the stoechiometric coefficient is negative. The reaction rate constant $r$ is the rate at which the reaction happens which usually is temperature-dependent and the order $O_i$ for a species $i$ specifies to which order the reaction depends on the density of that species. Positive orders mean that the reaction is faster the more density of this species is present, for negative orders the reaction slows down with higher density.
In general, this process can be written as $\Gamma = r \left[ A \right]^{O_A} \left[ B \right]^{O_B} \left[ C \right]^{O_C}$, where $\Gamma$ is known as the reaction rate. This is sometimes also called the [rate equation](https://en.wikipedia.org/wiki/Rate_equation).

For our specific simulation this means that all stoechiometric coefficients are $-1$ for the educts and $1$ for the product. The order of the educts is $1$ and the order of the product is $0$. This means that the more amount of both educts is present, the more will react and the amount of product present won't influence the amount of product that will be produced.

In [ ]:
REACTION_RATE_CONSTANT = 0.1

EDUCT_COEFFS = [1.0, 1.0]
PRODUCT_COEFFS = [1]

Now that we specified all coefficients for the reaction, we can setup all the necessary objects for the simulations.

We create each involved species and directly specify their boundary-conditions for the domain-boundaries. We set the initial density of the species to 0 and also add Dirichlet boundary conditions of zero density at both the inlet and the outlet of the system.

In [ ]:
educt_species = []
product_species = []
reactants = []
for coeff in EDUCT_COEFFS:
    species = espressomd.electrokinetics.EKSpecies(
        lattice=lattice, density=0.0, kT=KT,
        diffusion=DIFFUSION_COEFFICIENT, valency=0.0,
        advection=True, friction_coupling=True,
        ext_efield=[0., 0., 0.], tau=TAU)
    system.ekcontainer.add(species)
    reactants.append(
        espressomd.electrokinetics.EKReactant(
            ekspecies=species,
            stoech_coeff=-coeff,
            order=coeff))
    educt_species.append(species)
    species[0,:,:].density_boundary = espressomd.electrokinetics.DensityBoundary(0.0)
    species[-1,:,:].density_boundary = espressomd.electrokinetics.DensityBoundary(0.0)

for coeff in PRODUCT_COEFFS:
    species = espressomd.electrokinetics.EKSpecies(
        lattice=lattice, density=0.0, diffusion=DIFFUSION_COEFFICIENT,
        kT=KT, valency=0.0, advection=True, friction_coupling=True,
        ext_efield=[0., 0., 0.], tau=TAU)
    system.ekcontainer.add(species)
    reactants.append(
        espressomd.electrokinetics.EKReactant(
            ekspecies=species,
            stoech_coeff=coeff,
            order=0.0))
    product_species.append(species)
    species[0,:,:].density_boundary = espressomd.electrokinetics.DensityBoundary(0.0)
    species[-1,:,:].density_boundary = espressomd.electrokinetics.DensityBoundary(0.0)

# Exercise:
- Create an instance of [`EKBulkReaction`](https://espressomd.github.io/doc/espressomd.html#espressomd.electrokinetics.EKBulkReaction) using the previously setup `reactants` and activate the reaction by adding it to [`system.ekcontainer.reactions`](https://espressomd.github.io/doc/espressomd.html#espressomd.electrokinetics.EKContainer.reactions).


In [ ]:
reaction = espressomd.electrokinetics.EKBulkReaction(
    reactants=reactants, coefficient=REACTION_RATE_CONSTANT, lattice=lattice, tau=TAU)

system.ekcontainer.reactions.add(reaction)

The next thing to add to the system is the cylindrical boundaries, which act as the boundaries for the Kármán vorticies to form. These are placed close to the inlet of the system and also act as impenetrable boundaries for the species.
Since espresso uses periodic-boundary conditions, we need to add a total of three cylinders to the system, which will form two complete cylinders in the periodic system.

In [ ]:
cylinder_centers = [
    [BOX_L[0] // 10, 0, 1],
    [BOX_L[0] // 10, BOX_L[1] // 2, 1],
    [BOX_L[0] // 10, BOX_L[1], 1],    
]

shape_cylinder = []
for cylinder_center in cylinder_centers:
    shape_cylinder.append(espressomd.shapes.Cylinder(
        center=cylinder_center,
        axis=[0, 0, 1],
        length=BOX_L[2],
        radius=OBSTACLE_RADIUS,
    ))

for shape in shape_cylinder:
  lb.add_boundary_from_shape(shape)
  for spec in (*educt_species, *product_species):
    spec.add_boundary_from_shape(shape, value=[0,0,0], boundary_type=espressomd.electrokinetics.FluxBoundary)
    spec.add_boundary_from_shape(shape, value=0., boundary_type=espressomd.electrokinetics.DensityBoundary)

Up to this point there is no species present anywhere in the system and also no way for it to enter the system. Since the reaction is irreversible in our setup, we need to introduce some density of both the educt species to the system.
For that we setup two additional Dirichlet boundary conditions in the domain, where we fix the species density to a constant, non-zero value. This is done in such a way, that these two sources are spatially separated and only upon diffusion of the species through the channel, they will be able to react with each other. 

In [ ]:
source_boundary = espressomd.electrokinetics.DensityBoundary(10.0)
source_x_pos = BOX_L[1] // 20

educt_species[0][source_x_pos,BOX_L[1]//4-1:BOX_L[1]//4+1,:].density_boundary = source_boundary
educt_species[1][source_x_pos,3*(BOX_L[1]//4)-1:3*(BOX_L[1]//4)+1,:].density_boundary = source_boundary

With this, the system is now finally complete and we can start the integration. To see the system evolve, we will render a movie from the timeseries of the system. For that we have to setup some helper functions for the plotting, which are beyond the scope of this tutorial.

In [ ]:
VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

# set ignore 'divide' and 'invalid' errors
# these occur when plotting the flowfield containing a zero velocity
np.seterr(divide='ignore', invalid='ignore')

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with tempfile.NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            with open(f.name, "rb") as g:
                video = g.read()
        anim._encoded_video = base64.b64encode(video).decode('ascii')
        plt.close(anim._fig)
    return VIDEO_TAG.format(anim._encoded_video)

animation.Animation._repr_html_ = anim_to_html

In [ ]:
box_width = lattice.shape[1]
box_height = lattice.shape[0]

boundary_mask = lb[:, :, 0].boundary != None

progress_bar = tqdm.tqdm(total=TOTAL_FRAMES)

cmap = mpl.cm.get_cmap("viridis").copy()
cmap.set_bad(color="gray")
cmap_quiver = mpl.cm.get_cmap("binary").copy()
cmap_quiver.set_bad(color="gray")

# setup figure and prepare axes
fig = plt.figure(figsize=(20, 10))
gs = fig.add_gridspec(1, 4, wspace=0.1)
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1], sharey=ax1)
ax3 = plt.subplot(gs[2], sharey=ax1)
ax4 = plt.subplot(gs[3], sharey=ax1)

# set the background color for the quiver plot
bg_colors = np.copy(boundary_mask).astype(float)
bg_colors[boundary_mask] = np.NaN
ax4.imshow(bg_colors, cmap=cmap_quiver)

for ax, title in zip(
    [ax1, ax2, ax3, ax4],
    ["educt 1", "educt 2", "product", "fluid velocity"]
):
    ax.set_title(title)
    ax.set_xlim((0, box_width))
    ax.set_ylim((0, box_height))

# create meshgrid for quiver plot
xs = np.arange(box_width)
ys = np.arange(box_height)
X, Y = np.meshgrid(xs, ys)

flowfield = lb[:, :, 0].velocity
quiver = ax4.quiver(X, Y, flowfield[..., 1], flowfield[..., 0], scale=100)

def draw_frame(frame):
    system.integrator.run(100)
    
    flowfield = np.copy(lb[:, :, 0].velocity)
    e1 = np.copy(educt_species[0][:, :, 0].density)
    e2 = np.copy(educt_species[1][:, :, 0].density)
    p = np.copy(product_species[0][:, :, 0].density)
    
    # apply the mask for the boundary
    e1[boundary_mask] = np.NaN
    e2[boundary_mask] = np.NaN
    p[boundary_mask] = np.NaN
    flowfield[boundary_mask] = np.NaN


    ax1.imshow(e1, cmap=cmap)
    ax2.imshow(e2, cmap=cmap)
    ax3.imshow(p, cmap=cmap)
    quiver.set_UVC(flowfield[..., 1], flowfield[..., 0])
    
    progress_bar.update()
    

animation.FuncAnimation(fig, draw_frame, frames=range(TOTAL_FRAMES), interval=300)

Looking at the movie of the species densities one can see that the fluid flow is advecting the educt species from their source locations past the cylinders into the system, which is when they start to mix and react, such that the product will form. The density of the product is then increasing towards the outflow-location of the channel, at which it will then be deleted because of our zero-density boundary condition.